In [1]:
from IPython.display import Image, display

import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv

load_dotenv()

config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    }
]

import json
config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

gpt_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

In [2]:
import os
import json
import arxiv
import autogen
import fpdf
from datetime import datetime
from typing import List, Dict, Any, Optional

# Create a UserProxyAgent with a better configuration for function execution
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message="A human user who needs a technical review document.",
    code_execution_config={
        "last_n_messages": 3, 
        "work_dir": "paper_review",
        "use_docker": False  # Set to True if you want to use Docker for isolation
    },
    human_input_mode="NEVER"
)

# Researcher agent with improved system message
researcher = autogen.AssistantAgent(
    name="Researcher",
    system_message="""You are a researcher specialized in finding relevant research papers.
    You're expert at retrieving and analyzing scientific publications.
    When asked to search for papers, ALWAYS use the provided search_arxiv function by writing 
    code to call this function with appropriate parameters.
    Format your findings as a structured report with clear sections and paper summaries.
    """,
    llm_config={"config_list": gemini_config_list}
)

# Writer agent
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="""You are a technical writer specialized in creating comprehensive review documents.
    You take research papers and organize them into a structured review document.
    Your reviews include:
    1. Executive summary
    2. Introduction to the field
    3. Analysis of current research directions
    4. Detailed summaries of key papers
    5. Comparison of methodologies
    6. Future research directions
    7. Conclusion
    8. References (in IEEE format)
    Your writing is clear, precise, and technical but accessible.""",
    llm_config={"config_list": gemini_config_list}
)

# PDF creator agent
pdf_creator = autogen.AssistantAgent(
    name="PDFCreator",
    system_message="""You are an expert in creating PDF documents using Python.
    You take the final review document content and format it into a professional PDF.
    You use the fpdf library to create well-structured, easy-to-read PDFs.
    ALWAYS use the provided create_pdf function by writing code to call this function 
    with appropriate content structure.""",
    llm_config={"config_list": gemini_config_list}
)

# Define the functions that will be registered and made available to the agents

def search_arxiv(topic: str, max_results: int = 15) -> List[Dict[Any, Any]]:
    """
    Search arXiv for papers on a given topic and return results. Does not require any API_KEY.
    
    Args:
        topic: Search query string
        max_results: Maximum number of results to return
        
    Returns:
        List of dictionaries containing paper information
    """
    print(f"Searching arXiv for: {topic}")
    client = arxiv.Client()
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )
    
    results = []
    for paper in client.results(search):
        paper_dict = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "published": paper.published.strftime("%Y-%m-%d"),
            "url": paper.pdf_url,
            "arxiv_id": paper.get_short_id(),
            "categories": paper.categories
        }
        results.append(paper_dict)
    
    print(f"Found {len(results)} papers on arXiv")
    return results

def create_pdf(content: Dict[str, str], filename: str = "technical_review.pdf") -> str:
    """
    Create a PDF document from the content dictionary.
    
    Args:
        content: Dictionary with structured content for the PDF
        filename: Output filename for the PDF
        
    Returns:
        Path to the created PDF file
    """
    print(f"Creating PDF: {filename}")
    pdf = fpdf.FPDF()
    
    # Title page
    pdf.add_page()
    pdf.set_font("Arial", "B", 24)
    pdf.cell(0, 20, content["title"], ln=True, align="C")
    pdf.set_font("Arial", "", 12)
    pdf.cell(0, 10, f"Created: {datetime.now().strftime('%Y-%m-%d')}", ln=True, align="C")
    
    # Table of contents
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "Table of Contents", ln=True)
    pdf.set_font("Arial", "", 12)
    
    y_position = pdf.get_y()
    for section in ["executive_summary", "introduction", "current_research", "key_papers", 
                   "methodology_comparison", "future_directions", "conclusion", "references"]:
        if section in content:
            section_title = section.replace("_", " ").title()
            pdf.cell(0, 8, f"{section_title}", ln=True)
    
    # Add content sections
    sections = [
        ("Executive Summary", content.get("executive_summary", "")),
        ("Introduction", content.get("introduction", "")),
        ("Current Research Directions", content.get("current_research", "")),
        ("Key Papers", content.get("key_papers", "")),
        ("Methodology Comparison", content.get("methodology_comparison", "")),
        ("Future Research Directions", content.get("future_directions", "")),
        ("Conclusion", content.get("conclusion", "")),
        ("References", content.get("references", ""))
    ]
    
    for title, text in sections:
        if text:
            pdf.add_page()
            pdf.set_font("Arial", "B", 16)
            pdf.cell(0, 10, title, ln=True)
            pdf.set_font("Arial", "", 12)
            
            # Split text into paragraphs and add to PDF
            paragraphs = text.split("\n\n")
            for para in paragraphs:
                pdf.multi_cell(0, 8, para)
                pdf.ln(4)
    
    # Save the PDF
    filepath = os.path.abspath(filename)
    pdf.output(filename)
    print(f"PDF created at: {filepath}")
    return filepath

# Register the functions with the user proxy agent
user_proxy.register_function(
    function_map={
        "search_arxiv": search_arxiv,
        "create_pdf": create_pdf
    }
)

# Create a function to run the entire workflow
def run_review_workflow(topic: str) -> str:
    """
    Execute the complete workflow to generate a technical review document.
    
    Args:
        topic: The research topic to generate a review for
        
    Returns:
        Path to the generated PDF
    """
    # Create the working directory
    os.makedirs("paper_review", exist_ok=True)
    
    # Create a group chat
    groupchat = autogen.GroupChat(
        agents=[user_proxy, researcher, writer, pdf_creator],
        messages=[]
    )
    
    # Create a group chat manager WITH an LLM config (this was missing before)
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": gemini_config_list}  # Added this line to fix the error
    )
    
    # Start the chat with clear instructions
    user_proxy.initiate_chat(
        manager,
        message=f"""
        I need a comprehensive technical review document on "{topic}".
        
        Follow these steps exactly:
        
        1. Researcher: Search for the most relevant papers on this topic using the search_arxiv function.
           Do not use code that depends on extarnal API_KEYs.
           For example:
           ```python
           papers = search_arxiv("{topic}", max_results=15)
           # Analyze the papers here
           ```
           
        2. Researcher: Analyze the results and identify the 5-7 most important papers.
        
        3. Writer: Create a comprehensive technical review based on these papers.
        
        4. PDFCreator: Use the create_pdf function to generate a PDF with the following structure:
           ```python
           pdf_path = create_pdf({{
               "title": "Technical Review: {topic}",
               "executive_summary": "...",
               "introduction": "...",
               "current_research": "...",
               "key_papers": "...",
               "methodology_comparison": "...",
               "future_directions": "...",
               "conclusion": "...",
               "references": "..."
           }}, filename="{topic.replace(' ', '_')}_review.pdf")
           ```
           
        The final output should be a PDF file saved in the working directory.
        """
    )
    
    # Return the expected filename
    return f"{topic.replace(' ', '_')}_review.pdf"


# Alternative implementation using direct agent interactions instead of group chat
def run_sequential_workflow(topic: str) -> str:
    """
    Execute the workflow in a more sequential manner with direct agent interactions.
    This approach is more reliable when you encounter issues with the GroupChat.
    
    Args:
        topic: The research topic to generate a review for
        
    Returns:
        Path to the generated PDF
    """
    # Create the working directory
    os.makedirs("paper_review", exist_ok=True)
    
    # Step 1: Have researcher find papers
    user_proxy.initiate_chat(
        researcher,
        message=f"""
        I need you to find the most relevant papers on "{topic}".
        Use the search_arxiv function to search for papers.
        
        Example code:
        ```python
        papers = search_arxiv("{topic}", max_results=15)
        # Now analyze these papers to find the most important ones
        ```
        
        After finding the papers, select the 5-7 most important ones and provide a brief summary of each.
        """
    )
    
    # Step 2: Get the writer to create a review
    research_results = user_proxy.last_message()
    
    user_proxy.initiate_chat(
        writer,
        message=f"""
        Based on the following research findings, create a comprehensive technical review on "{topic}".
        
        Research findings:
        {research_results}
        
        Your review should include:
        1. Executive summary
        2. Introduction to the field
        3. Analysis of current research directions
        4. Detailed summaries of key papers
        5. Comparison of methodologies
        6. Future research directions
        7. Conclusion
        8. References (in IEEE format)
        """
    )
    
    # Step 3: Have PDF creator generate the PDF
    review_content = user_proxy.last_message()
    
    user_proxy.initiate_chat(
        pdf_creator,
        message=f"""
        Create a PDF document from the following technical review on "{topic}".
        
        Review content:
        {review_content}
        
        Use the create_pdf function to generate a PDF. Example:
        ```python
        pdf_path = create_pdf({{
            "title": "Technical Review: {topic}",
            "executive_summary": "Executive summary text...",
            "introduction": "Introduction text...",
            "current_research": "Current research text...",
            "key_papers": "Key papers text...",
            "methodology_comparison": "Methodology comparison text...",
            "future_directions": "Future directions text...",
            "conclusion": "Conclusion text...",
            "references": "References text..."
        }}, filename="{topic.replace(' ', '_')}_review.pdf")
        ```
        
        Make sure to extract the appropriate content from the review for each section.
        """
    )
    
    # Return the expected filename
    return f"{topic.replace(' ', '_')}_review.pdf"

# Usage example
if __name__ == "__main__":
    # Set your OpenAI API key as an environment variable before running
    # os.environ["OPENAI_API_KEY"] = "your-api-key"
    
    topic = "quantum machine learning"
    
    try:
        # Try the group chat approach first
        pdf_path = run_review_workflow(topic)
        print(f"Group chat workflow completed. Review document should be at: {pdf_path}")
    except Exception as e:
        print(f"Group chat workflow failed with error: {e}")
        print("Trying sequential approach instead...")
        # Fall back to the sequential approach if group chat fails
        pdf_path = run_sequential_workflow(topic)
        print(f"Sequential workflow completed. Review document should be at: {pdf_path}")

User (to chat_manager):


        I need a comprehensive technical review document on "quantum machine learning".
        
        Follow these steps exactly:
        
        1. Researcher: Search for the most relevant papers on this topic using the search_arxiv function.
           Do not use code that depends on extarnal API_KEYs.
           For example:
           ```python
           papers = search_arxiv("quantum machine learning", max_results=15)
           # Analyze the papers here
           ```
           
        2. Researcher: Analyze the results and identify the 5-7 most important papers.
        
        3. Writer: Create a comprehensive technical review based on these papers.
        
        4. PDFCreator: Use the create_pdf function to generate a PDF with the following structure:
           ```python
           pdf_path = create_pdf({
               "title": "Technical Review: quantum machine learning",
               "executive_summary": "...",
               "introdu